In [1]:
import os
import json
import time
import shutil
import rasterio
import urllib.request
import shapely
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt
from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

from utils import crop_raster, merge_tiles, stitch_tiles, get_tiles

import warnings
warnings.filterwarnings('ignore')


In [2]:
START_DATE = os.getenv('START_DATE', '2016-05-01')
END_DATE = os.getenv('END_DATE', '2023-06-30')
AOI = os.getenv(
    'AOI', 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))')
# AOI = os.getenv('AOI', 'MULTIPOLYGON (((-110.49367408906883 39.1005200348037, -110.20673076923077 39.1005200348037, -110.16573886639677 38.41732165423689, -110.43901821862349 38.41732165423689, -110.49367408906883 39.1005200348037)))')

SATELLITE_CACHE_FOLDER = os.getenv('SENTINEL2_CACHE', os.path.join(
    "code", "sentinel_cache", "landcover_dataset"))
OUTPUT_FOLDER = os.getenv(
    'OUTPUT_FOLDER', os.path.join("code", "results", "landcover"))

landcovers_grid = gpd.read_file('landcovers_grid.geojson')

In [ ]:
AOI='MULTIPOLYGON (((14.66114877031896 48.58274154385973, 14.72399509478079 48.58655041200893, 14.72617159086605 48.53050563781352, 14.6589722742337 48.49513757642807, 14.66114877031896 48.58274154385973)))'

In [4]:
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

aoi = gpd.read_file(aoi_filename)


In [5]:
s2overlap = Sentinel2Overlap(aoi_path=aoi_filename)
overlap_tiles = landcovers_grid[landcovers_grid.intersects(polygon)].name.values.tolist()

start_year = int(START_DATE.split("-")[0])
end_year = int(END_DATE.split("-")[0])

date_range = range(2017, 2022)

if end_year in date_range:
    year = end_year
elif start_year in date_range:
    year = start_year
else:
    year = max(list(date_range))


In [6]:
year


2019

In [7]:
files = []
landcover_dataset_path = os.path.join(
    SATELLITE_CACHE_FOLDER, "landcover_dataset")
os.makedirs(landcover_dataset_path, exist_ok=True)

for tile_i in overlap_tiles:
    tile_i = tile_i if len(tile_i) < 4 else tile_i[:3]
    tile_url = f"https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year}/{tile_i}_{year}0101-{year+1}0101.tif"
    path = os.path.join(landcover_dataset_path, os.path.basename(tile_url))
    if not os.path.exists(path):
        urllib.request.urlretrieve(tile_url, path)
    files.append(path)

files


['/input/SENTINEL2_CACHE/landcover_dataset/32T_20190101-20200101.tif',
 '/input/SENTINEL2_CACHE/landcover_dataset/33T_20190101-20200101.tif']

In [8]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

class_colors = {
    1: "65,155,223",
    2: "57,125,73",
    4: "122,135,198",
    5: "228,150,53",
    7: "196,50,27",
    8: "165,155,143",
    9: "168,235,255",
    10: "97,97,97",
    11: "227,226,195"
}

In [9]:
if len(files) > 1:
    try:
        raster_path = merge_tiles(
            files, os.path.join(SATELLITE_CACHE_FOLDER, "aoi_raster.tif"))
    except Exception:
        raster_path = stitch_tiles(
            files, os.path.join(SATELLITE_CACHE_FOLDER, "aoi_raster.tif"))
else:
    raster_path = files[0]


Start merging to /input/SENTINEL2_CACHE/aoi_raster.tif
Copying color table from /input/SENTINEL2_CACHE/landcover_dataset/32T_20190101-20200101.tif to new file.
Creating output file that is 102679P x 89839L.
Processing /input/SENTINEL2_CACHE/landcover_dataset/32T_20190101-20200101.tif [1/2] : 0Using internal nodata values (e.g. 0) for image /input/SENTINEL2_CACHE/landcover_dataset/32T_20190101-20200101.tif.
Copying nodata values from source /input/SENTINEL2_CACHE/landcover_dataset/32T_20190101-20200101.tif to destination /input/SENTINEL2_CACHE/aoi_raster.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Processing /input/SENTINEL2_CACHE/landcover_dataset/33T_20190101-20200101.tif [2/2] : 0Using internal nodata values (e.g. 0) for image /input/SENTINEL2_CACHE/landcover_dataset/33T_20190101-20200101.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
300.51434230804443 seconds for merging 2 images in to /input/SENTINEL2_CACHE/aoi_raster.tif


In [10]:
raster_path = crop_raster(
    raster_path,
    aoi_filename,
    raster_path.replace(".tif", "_crop.tif"),
    additional_meta={
        "START_DATE": f"{year}-01-01",
        "END_DATE": f"{year+1}-01-01",
        "NAME": "Landcover classification"})

with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile


In [11]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0, 0, 0, 1)

labels = []

for label, name in enumerate(class_names):
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    if class_area != 0:
        labels.append({
            "color": class_colors[name],
            "name": class_names[name],
            "area": class_area
        })
    else:
        labels.append({
            "color": class_colors[name],
            "name": "Other" if name not in class_names.keys() else class_names[name],
            "area": 0.0
        })
labels = json.dumps(labels)
labels


'[{"color": "65,155,223", "name": "Water", "area": 0.1119}, {"color": "57,125,73", "name": "Trees", "area": 0.2864}, {"color": "122,135,198", "name": "Flooded vegetation", "area": 48.6103}, {"color": "228,150,53", "name": "Crops", "area": 0.0}, {"color": "196,50,27", "name": "Built Area", "area": 0.0}, {"color": "165,155,143", "name": "Bare ground", "area": 0.0}, {"color": "168,235,255", "name": "Snow/Ice", "area": 0.0}, {"color": "97,97,97", "name": "Clouds", "area": 0.004}, {"color": "227,226,195", "name": "Rangeland", "area": 0.3059}]'

In [12]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

NUM_CLASSES = 11
nodata = 0
mask = mask.astype(bool)
scaled = img.astype(np.float32) / NUM_CLASSES
scaled = np.zeros((*img.shape, 3), dtype=np.uint8)
for label, color in class_colors.items():
    color_mask = img == label
    scaled[color_mask] = np.array(color.split(','), dtype=np.uint8)
    
scaled[mask[:, :, np.newaxis] & (scaled == 0)] += 1
scaled = np.clip(scaled, 0, 255).astype(np.uint8)
# Set pixels with invalid pixels to new nodata value
scaled[~mask] = nodata
# # Set pixels with background class(0) to new nodata value
scaled[img == 0] = nodata

profile.update(
    count=3,
    nodata=nodata,
    compress='lzw'
)
colored_tif = os.path.join(OUTPUT_FOLDER, f"{START_DATE}_{END_DATE}.tif")

with rasterio.open(colored_tif, 'w', **profile) as dst:
    dst.update_tags(start_date=START_DATE,
                    end_date=END_DATE,
                    labels=labels,
                    name="Landcover")
    for i in range(scaled.shape[-1]):
        dst.write(scaled[:, :, i], indexes=i+1)

In [14]:
os.remove(raster_path)
os.remove(aoi_filename)
